# 🎤 Piper TTS - Training Completo da Zero

## 📋 Panoramica
Questo notebook permette di addestrare un modello Piper TTS completamente da zero.

**Differenze con Fine-Tuning:**
- ✅ Training completo: crei un nuovo modello da zero
- ✅ Massimo controllo su architettura e hyperparameters
- ✅ Ideale per lingue/accenti non supportati
- ⏱️ Tempo: ~12-16 ore (vs 8-12h del fine-tuning)

**Requisiti:**
- Google Colab con GPU (T4 o superiore)
- Dataset audio pulito (min 30 minuti, consigliato 2+ ore)
- File di trascrizione accurati

---

## 🔧 Step 1: Setup Ambiente

In [ ]:
# Verifica GPU
!nvidia-smi

print("\n✅ Se vedi info GPU sopra, sei pronto!")
print("❌ Se errore: Runtime > Change runtime type > GPU")

In [ ]:
# Installazione dipendenze
print("📦 Installazione Piper Training...")

!pip install -q piper-phonemize
!pip install -q onnxruntime
!pip install -q espeak-ng

# Clona repository Piper
!git clone https://github.com/rhasspy/piper.git
%cd piper/src/python

# Installa requirements
!pip install -q -r requirements.txt
!pip install -q -e .

%cd /content
print("\n✅ Installazione completata!")

## 📁 Step 2: Preparazione Dataset

### Struttura Dataset Richiesta:
```
my_dataset/
├── wavs/              # File audio WAV (16kHz, mono)
│   ├── audio_001.wav
│   ├── audio_002.wav
│   └── ...
└── metadata.csv       # Trascrizioni
```

### Formato metadata.csv:
```
audio_001|Questa è la prima frase.
audio_002|Questa è la seconda frase.
```

In [ ]:
# Monta Google Drive (se il dataset è lì)
from google.colab import drive
drive.mount('/content/drive')

# Oppure carica manualmente i file
# from google.colab import files
# uploaded = files.upload()

In [ ]:
# Configura percorsi
DATASET_DIR = "/content/drive/MyDrive/my_dataset"  # ⚠️ MODIFICA QUI
OUTPUT_DIR = "/content/piper_output"

!mkdir -p {OUTPUT_DIR}

# Verifica dataset
import os

assert os.path.exists(DATASET_DIR), f"❌ Dataset non trovato in {DATASET_DIR}"
assert os.path.exists(f"{DATASET_DIR}/metadata.csv"), "❌ metadata.csv mancante"
assert os.path.exists(f"{DATASET_DIR}/wavs"), "❌ Cartella wavs/ mancante"

# Conta file audio
num_wavs = len([f for f in os.listdir(f"{DATASET_DIR}/wavs") if f.endswith('.wav')])
print(f"\n✅ Dataset trovato: {num_wavs} file audio")

if num_wavs < 100:
    print("⚠️ Warning: Pochi file audio. Consigliati almeno 500-1000 per buoni risultati.")

## 🔍 Step 3: Validazione Dataset

In [ ]:
# Verifica formato audio
import librosa
import pandas as pd

# Carica metadata
metadata = pd.read_csv(f"{DATASET_DIR}/metadata.csv", 
                       sep='|', 
                       header=None, 
                       names=['filename', 'text'])

print(f"📊 Statistiche Dataset:")
print(f"   Totale frasi: {len(metadata)}")
print(f"   Lunghezza media testo: {metadata['text'].str.len().mean():.0f} caratteri")

# Verifica alcuni file audio
sample_files = metadata['filename'].head(5).tolist()

for fname in sample_files:
    wav_path = f"{DATASET_DIR}/wavs/{fname}.wav"
    if os.path.exists(wav_path):
        y, sr = librosa.load(wav_path, sr=None)
        duration = len(y) / sr
        print(f"   {fname}: {sr}Hz, {duration:.2f}s")
        
        if sr != 22050:
            print(f"      ⚠️ Sample rate non ottimale. Consigliato: 22050Hz")
    else:
        print(f"   ❌ File mancante: {wav_path}")

print("\n✅ Validazione completata!")

## 🎛️ Step 4: Configurazione Training

In [ ]:
# Crea file di configurazione
import json

config = {
    "audio": {
        "sample_rate": 22050,
        "max_wav_value": 32767.0,
        "filter_length": 1024,
        "hop_length": 256,
        "win_length": 1024
    },
    "model": {
        "name": "vits",
        "hidden_channels": 192,
        "inter_channels": 192,
        "filter_channels": 768,
        "n_heads": 2,
        "n_layers": 6,
        "kernel_size": 3,
        "p_dropout": 0.1
    },
    "training": {
        "epochs": 10000,
        "learning_rate": 0.0002,
        "batch_size": 16,
        "log_interval": 100,
        "save_interval": 1000,
        "num_workers": 4
    },
    "dataset": {
        "path": DATASET_DIR,
        "text_cleaners": ["english_cleaners"],  # ⚠️ Modifica per la tua lingua
        "language": "en-us"  # ⚠️ Modifica codice lingua
    }
}

# Salva configurazione
config_path = f"{OUTPUT_DIR}/config.json"
with open(config_path, 'w') as f:
    json.dump(config, f, indent=2)

print(f"✅ Configurazione salvata in {config_path}")
print("\n📝 Parametri principali:")
print(f"   - Epochs: {config['training']['epochs']}")
print(f"   - Batch size: {config['training']['batch_size']}")
print(f"   - Learning rate: {config['training']['learning_rate']}")

## 🚀 Step 5: Avvio Training

In [ ]:
# Preprocessing del dataset
print("🔄 Preprocessing dataset...")

!python /content/piper/src/python/piper_train/preprocess.py \
    --input-dir {DATASET_DIR} \
    --output-dir {OUTPUT_DIR}/preprocessed \
    --language {config['dataset']['language']} \
    --sample-rate {config['audio']['sample_rate']}

print("\n✅ Preprocessing completato!")

In [ ]:
# Training (questo richiederà diverse ore)
print("🎯 Avvio training... (questo richiederà 12-16 ore)")
print("💡 Puoi monitorare i progressi nel log sotto.\n")

%cd /content/piper/src/python

!python -m piper_train \
    --dataset-dir {OUTPUT_DIR}/preprocessed \
    --output-dir {OUTPUT_DIR}/checkpoints \
    --config {config_path} \
    --restore-checkpoint  # Riprende da ultimo checkpoint se interrotto

%cd /content
print("\n🎉 Training completato!")

## 📊 Step 6: Monitoraggio Training (Opzionale)

In [ ]:
# Visualizza ultimi checkpoint
import glob

checkpoints = sorted(glob.glob(f"{OUTPUT_DIR}/checkpoints/*.pt"))
print(f"📁 Checkpoint trovati: {len(checkpoints)}\n")

for ckpt in checkpoints[-5:]:  # Ultimi 5
    size_mb = os.path.getsize(ckpt) / (1024*1024)
    print(f"   {os.path.basename(ckpt)} ({size_mb:.1f}MB)")

## 🎵 Step 7: Export Modello Finale

In [ ]:
# Converti checkpoint PyTorch in ONNX (formato Piper)
print("📦 Export modello in formato ONNX...")

# Trova ultimo checkpoint
latest_checkpoint = sorted(glob.glob(f"{OUTPUT_DIR}/checkpoints/*.pt"))[-1]
print(f"   Usando: {os.path.basename(latest_checkpoint)}")

%cd /content/piper/src/python

!python -m piper_train.export_onnx \
    {latest_checkpoint} \
    {OUTPUT_DIR}/model.onnx

%cd /content
print("\n✅ Modello esportato: model.onnx")

## 🧪 Step 8: Test Modello

In [ ]:
# Test sintetizzazione
from IPython.display import Audio
import subprocess

test_text = "Questo è un test del modello addestrato."  # ⚠️ Modifica testo
output_wav = f"{OUTPUT_DIR}/test_output.wav"

print(f"🎤 Sintetizzo: '{test_text}'...\n")

# Usa Piper per generare audio
cmd = [
    "piper",
    "--model", f"{OUTPUT_DIR}/model.onnx",
    "--output_file", output_wav
]

subprocess.run(cmd, input=test_text.encode('utf-8'))

print("✅ Audio generato! Ascolta sotto:\n")
Audio(output_wav)

## 💾 Step 9: Download Modello

In [ ]:
# Crea archivio con modello e config
import shutil

print("📦 Creazione archivio finale...\n")

# Copia file necessari
model_package = f"{OUTPUT_DIR}/my_piper_model"
!mkdir -p {model_package}

shutil.copy(f"{OUTPUT_DIR}/model.onnx", model_package)
shutil.copy(config_path, model_package)

# Crea archivio ZIP
shutil.make_archive(model_package, 'zip', model_package)

print(f"✅ Modello pronto per il download!")
print(f"   Percorso: {model_package}.zip\n")

# Download
from google.colab import files
files.download(f"{model_package}.zip")

print("\n🎉 Training completato con successo!")

## 📝 Note Finali

### Prossimi Passi:
1. **Fine-Tuning**: Puoi ora usare questo modello come base per ulteriore fine-tuning
2. **Testing**: Prova il modello con frasi diverse per valutare la qualità
3. **Ottimizzazione**: Se i risultati non sono ottimali:
   - Aumenta il dataset (più audio = migliori risultati)
   - Aumenta epochs
   - Modifica learning rate

### Troubleshooting:
- **OOM (Out of Memory)**: Riduci batch_size nel config
- **Audio distorto**: Verifica che tutti i WAV siano 22050Hz mono
- **Training lento**: Assicurati di usare GPU T4 o superiore

### Risorse:
- 📖 [Piper Documentation](https://github.com/rhasspy/piper)
- 💬 [Piper Discussion](https://github.com/rhasspy/piper/discussions)

---

**Buon training! 🚀**